# Starting to explore daily train data

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import datetime
import time
from dateutil.relativedelta import relativedelta

# Hack for using import with relative path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from fetchTrainData import fetch_train_data
from fetchWeatherData import fetch_weather_data, save_weather_data, open_weather_data

weather_data = {}

def get_weather(stop_):
    tag = stop_['station.shortCode']
    if type(tag) == str:
        lat = stop_['station.location'][0]
        lon = stop_['station.location'][1]
        if tag not in weather_data:
            print("FETCHING DATA FOR: " + tag)
            weather_data[tag] = fetch_weather_data(unix_time, lat, lon)['list'][0]['main']['temp']

            
def extract_time_tables(train_stops_):
    print("TRAIN NUMBER " + str(train_stops_.name) + ", " + str(float(train_stops_.name) / number_of_trains))
    train_stops = pd.json_normalize(train_stops_)
    to_return = train_stops['differenceInMinutes'].sum()
    train_stops = train_stops.drop(['differenceInMinutes'], axis=1)

    train_stops.apply(get_weather, axis=1)

    for stop in train_stops['station.shortCode'].unique():
        if type(stop) == str:
            if stop not in df.columns:
                df[stop] = None

            df[stop][train_stops_.name] = weather_data[stop]
        
    
    return to_return


loops = 12 # How many months data we want in the dataframe
number_of_trains = 500 # How many trains to fetch per month. Seems to be softlocked at about 1000, 500 is safe
all_data_frames = []
all_late_amounts = []


for i in range(loops):
    date = datetime.date.today() - relativedelta(months = i)
    unix_time = time.mktime(date.timetuple())
    print("LOOP " + str(i) + ", DATE: " + str(date))

    q = (
        """ 
        {
        """
          f'  trainsByDepartureDate(departureDate: \"{date}\",'
        """
        """
          f'  take: {number_of_trains},'
        """
            where: {
                and: [
                    {or: [{deleted: {unequals: null}}, 
                    {deleted: {equals: false}}]}, 
                    {cancelled: {equals: false}}, 
                    {operator: {shortCode: {equals: "vr"}}}
                    ]  
            }
          ) 
          {
            trainNumber
            departureDate
            commuterLineid
            trainType {
              name
              trainCategory {
                name
              }
            }
            timeTableRows {
            differenceInMinutes
              station {
                shortCode
                location
              }
            }
          }
        }
        """
        )

    result = fetch_train_data(q)
    recs = result['data']['trainsByDepartureDate']

    df = pd.json_normalize(recs)
    all_stops = pd.json_normalize(df['timeTableRows'])
    df = df.drop(['timeTableRows'], axis=1)

    weather_data = open_weather_data(date)
    how_much_late = all_stops.apply(extract_time_tables, axis=1)
    save_weather_data(date, weather_data)
    
    all_data_frames.append(df)
    all_late_amounts.append(how_much_late)
    
print(pd.concat(all_data_frames))
print(pd.concat(all_late_amounts))

LOOP 0, DATE: 2021-10-26
TRAIN NUMBER 0, 0.0
FETCHING DATA FOR: HKI
FETCHING DATA FOR: PSL
FETCHING DATA FOR: KÄP
FETCHING DATA FOR: OLK
FETCHING DATA FOR: PMK
FETCHING DATA FOR: ML
FETCHING DATA FOR: TNA
FETCHING DATA FOR: PLA
FETCHING DATA FOR: TKL
FETCHING DATA FOR: HKH
FETCHING DATA FOR: HVK
FETCHING DATA FOR: KVY
FETCHING DATA FOR: RKL
FETCHING DATA FOR: HNA
FETCHING DATA FOR: KRS
FETCHING DATA FOR: SAV
FETCHING DATA FOR: KE
FETCHING DATA FOR: KYT
FETCHING DATA FOR: HAA
FETCHING DATA FOR: KSU
FETCHING DATA FOR: MLÄ
FETCHING DATA FOR: SIP
FETCHING DATA FOR: HNN
FETCHING DATA FOR: HLT
FETCHING DATA FOR: LH
FETCHING DATA FOR: VLH
FETCHING DATA FOR: NSL
FETCHING DATA FOR: UKÄ
FETCHING DATA FOR: MKA
FETCHING DATA FOR: KA
FETCHING DATA FOR: NMÄ
FETCHING DATA FOR: SMÄ
FETCHING DATA FOR: KRA
FETCHING DATA FOR: KV
FETCHING DATA FOR: KUV
FETCHING DATA FOR: UTI
FETCHING DATA FOR: KTH
FETCHING DATA FOR: KPA
FETCHING DATA FOR: KJR
FETCHING DATA FOR: TA
FETCHING DATA FOR: LÄ
FETCHING DATA FOR: 

/tmp/ipykernel_23799/2372603998.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[stop] = None


TRAIN NUMBER 13, 0.026
TRAIN NUMBER 14, 0.028
FETCHING DATA FOR: ROI
FETCHING DATA FOR: MUL
FETCHING DATA FOR: KVU
FETCHING DATA FOR: TRV
FETCHING DATA FOR: TÖR
FETCHING DATA FOR: LLA
FETCHING DATA FOR: LI
FETCHING DATA FOR: KEM
FETCHING DATA FOR: SIM
FETCHING DATA FOR: MYS
FETCHING DATA FOR: II
FETCHING DATA FOR: HD
FETCHING DATA FOR: TUA
TRAIN NUMBER 15, 0.03
TRAIN NUMBER 16, 0.032
TRAIN NUMBER 17, 0.034
TRAIN NUMBER 18, 0.036
TRAIN NUMBER 19, 0.038
TRAIN NUMBER 20, 0.04
TRAIN NUMBER 21, 0.042
TRAIN NUMBER 22, 0.044
TRAIN NUMBER 23, 0.046
TRAIN NUMBER 24, 0.048
TRAIN NUMBER 25, 0.05
TRAIN NUMBER 26, 0.052
FETCHING DATA FOR: VS
FETCHING DATA FOR: LAI
FETCHING DATA FOR: TK
FETCHING DATA FOR: IKY
FETCHING DATA FOR: YST
TRAIN NUMBER 27, 0.054
TRAIN NUMBER 28, 0.056
TRAIN NUMBER 29, 0.058
TRAIN NUMBER 30, 0.06
TRAIN NUMBER 31, 0.062
TRAIN NUMBER 32, 0.064
TRAIN NUMBER 33, 0.066
TRAIN NUMBER 34, 0.068
TRAIN NUMBER 35, 0.07
TRAIN NUMBER 36, 0.072
TRAIN NUMBER 37, 0.074
TRAIN NUMBER 38, 0.07

FETCHING DATA FOR: KHK
FETCHING DATA FOR: HPL
FETCHING DATA FOR: VMO
FETCHING DATA FOR: PJM
FETCHING DATA FOR: MÄK
FETCHING DATA FOR: LPV
FETCHING DATA FOR: KIL
FETCHING DATA FOR: KEA
FETCHING DATA FOR: KNI
FETCHING DATA FOR: KVH
FETCHING DATA FOR: TRL
FETCHING DATA FOR: EPO
FETCHING DATA FOR: KLH
FETCHING DATA FOR: MAS
FETCHING DATA FOR: VKH
FETCHING DATA FOR: JRS
FETCHING DATA FOR: HEK
FETCHING DATA FOR: TOL
FETCHING DATA FOR: KKN
FETCHING DATA FOR: STI
FETCHING DATA FOR: IKO
FETCHING DATA FOR: KR
FETCHING DATA FOR: PKU
FETCHING DATA FOR: ERV
FETCHING DATA FOR: SLO
FETCHING DATA FOR: PO
FETCHING DATA FOR: PIK
FETCHING DATA FOR: KUT
TRAIN NUMBER 188, 0.376
TRAIN NUMBER 189, 0.378
TRAIN NUMBER 190, 0.38
TRAIN NUMBER 191, 0.382
TRAIN NUMBER 192, 0.384
TRAIN NUMBER 193, 0.386
TRAIN NUMBER 194, 0.388
TRAIN NUMBER 195, 0.39
TRAIN NUMBER 196, 0.392
TRAIN NUMBER 197, 0.394
TRAIN NUMBER 198, 0.396
TRAIN NUMBER 199, 0.398
TRAIN NUMBER 200, 0.4
TRAIN NUMBER 201, 0.402
TRAIN NUMBER 202, 0.404
TR

TRAIN NUMBER 424, 0.848
TRAIN NUMBER 425, 0.85
TRAIN NUMBER 426, 0.852
TRAIN NUMBER 427, 0.854
TRAIN NUMBER 428, 0.856
TRAIN NUMBER 429, 0.858
TRAIN NUMBER 430, 0.86
TRAIN NUMBER 431, 0.862
TRAIN NUMBER 432, 0.864
TRAIN NUMBER 433, 0.866
TRAIN NUMBER 434, 0.868
TRAIN NUMBER 435, 0.87
TRAIN NUMBER 436, 0.872
TRAIN NUMBER 437, 0.874
TRAIN NUMBER 438, 0.876
TRAIN NUMBER 439, 0.878
TRAIN NUMBER 440, 0.88
TRAIN NUMBER 441, 0.882
TRAIN NUMBER 442, 0.884
TRAIN NUMBER 443, 0.886
TRAIN NUMBER 444, 0.888
TRAIN NUMBER 445, 0.89
TRAIN NUMBER 446, 0.892
TRAIN NUMBER 447, 0.894
TRAIN NUMBER 448, 0.896
TRAIN NUMBER 449, 0.898
TRAIN NUMBER 450, 0.9
TRAIN NUMBER 451, 0.902
TRAIN NUMBER 452, 0.904
TRAIN NUMBER 453, 0.906
TRAIN NUMBER 454, 0.908
TRAIN NUMBER 455, 0.91
TRAIN NUMBER 456, 0.912
TRAIN NUMBER 457, 0.914
TRAIN NUMBER 458, 0.916
TRAIN NUMBER 459, 0.918
TRAIN NUMBER 460, 0.92
TRAIN NUMBER 461, 0.922
TRAIN NUMBER 462, 0.924
TRAIN NUMBER 463, 0.926
TRAIN NUMBER 464, 0.928
TRAIN NUMBER 465, 0.93
TR

...

FETCHING DATA FOR: HPJ
FETCHING DATA FOR: NVL
TRAIN NUMBER 129, 0.258
TRAIN NUMBER 130, 0.26
TRAIN NUMBER 131, 0.262
TRAIN NUMBER 132, 0.264
TRAIN NUMBER 133, 0.266
TRAIN NUMBER 134, 0.268
TRAIN NUMBER 135, 0.27
FETCHING DATA FOR: ONT
FETCHING DATA FOR: YLY
FETCHING DATA FOR: VNJ
FETCHING DATA FOR: VIH
FETCHING DATA FOR: HNV
FETCHING DATA FOR: SYR
FETCHING DATA FOR: VAR
FETCHING DATA FOR: HKO
FETCHING DATA FOR: SKÄ
TRAIN NUMBER 136, 0.272
TRAIN NUMBER 137, 0.274
TRAIN NUMBER 138, 0.276
TRAIN NUMBER 139, 0.278
FETCHING DATA FOR: PUN
FETCHING DATA FOR: LUS
FETCHING DATA FOR: REE
FETCHING DATA FOR: KIÄ
FETCHING DATA FOR: PKY
FETCHING DATA FOR: SL
TRAIN NUMBER 140, 0.28
TRAIN NUMBER 141, 0.282
TRAIN NUMBER 142, 0.284
TRAIN NUMBER 143, 0.286
TRAIN NUMBER 144, 0.288
TRAIN NUMBER 145, 0.29
TRAIN NUMBER 146, 0.292
TRAIN NUMBER 147, 0.294
TRAIN NUMBER 148, 0.296
TRAIN NUMBER 149, 0.298
TRAIN NUMBER 150, 0.3
TRAIN NUMBER 151, 0.302
FETCHING DATA FOR: NRM
FETCHING DATA FOR: KOH
FETCHING DATA FOR:

TRAIN NUMBER 342, 0.684
TRAIN NUMBER 343, 0.686
TRAIN NUMBER 344, 0.688
TRAIN NUMBER 345, 0.69
TRAIN NUMBER 346, 0.692
FETCHING DATA FOR: R934
FETCHING DATA FOR: ARO
FETCHING DATA FOR: YPY
FETCHING DATA FOR: VUS
TRAIN NUMBER 347, 0.694
TRAIN NUMBER 348, 0.696
TRAIN NUMBER 349, 0.698
TRAIN NUMBER 350, 0.7
TRAIN NUMBER 351, 0.702
FETCHING DATA FOR: VKT
FETCHING DATA FOR: MLK
FETCHING DATA FOR: VLM
TRAIN NUMBER 352, 0.704
TRAIN NUMBER 353, 0.706
TRAIN NUMBER 354, 0.708
TRAIN NUMBER 355, 0.71
TRAIN NUMBER 356, 0.712
TRAIN NUMBER 357, 0.714
TRAIN NUMBER 358, 0.716
TRAIN NUMBER 359, 0.718
TRAIN NUMBER 360, 0.72
TRAIN NUMBER 361, 0.722
TRAIN NUMBER 362, 0.724
TRAIN NUMBER 363, 0.726
TRAIN NUMBER 364, 0.728
TRAIN NUMBER 365, 0.73
TRAIN NUMBER 366, 0.732
TRAIN NUMBER 367, 0.734
TRAIN NUMBER 368, 0.736
TRAIN NUMBER 369, 0.738
TRAIN NUMBER 370, 0.74
TRAIN NUMBER 371, 0.742
TRAIN NUMBER 372, 0.744
TRAIN NUMBER 373, 0.746
TRAIN NUMBER 374, 0.748
TRAIN NUMBER 375, 0.75
TRAIN NUMBER 376, 0.752
TRAIN 

FETCHING DATA FOR: IKY
FETCHING DATA FOR: TK
FETCHING DATA FOR: LAI
FETCHING DATA FOR: VS
TRAIN NUMBER 22, 0.044
TRAIN NUMBER 23, 0.046
TRAIN NUMBER 24, 0.048
TRAIN NUMBER 25, 0.05
TRAIN NUMBER 26, 0.052
TRAIN NUMBER 27, 0.054
TRAIN NUMBER 28, 0.056
TRAIN NUMBER 29, 0.058
TRAIN NUMBER 30, 0.06
TRAIN NUMBER 31, 0.062
TRAIN NUMBER 32, 0.064
FETCHING DATA FOR: HJ
FETCHING DATA FOR: SPÄ
FETCHING DATA FOR: VHJ
FETCHING DATA FOR: HLS
FETCHING DATA FOR: KII
FETCHING DATA FOR: MR
FETCHING DATA FOR: MYT
FETCHING DATA FOR: LKL
FETCHING DATA FOR: OT
FETCHING DATA FOR: MI
FETCHING DATA FOR: HIR
FETCHING DATA FOR: KSA
FETCHING DATA FOR: HAU
FETCHING DATA FOR: LOL
FETCHING DATA FOR: PM
FETCHING DATA FOR: HPS
FETCHING DATA FOR: MRK
FETCHING DATA FOR: SNJ
FETCHING DATA FOR: SLN
FETCHING DATA FOR: ARL
FETCHING DATA FOR: KRM
FETCHING DATA FOR: KUO
TRAIN NUMBER 33, 0.066
FETCHING DATA FOR: KUOT
FETCHING DATA FOR: SOR
FETCHING DATA FOR: TOI
FETCHING DATA FOR: SIJ
FETCHING DATA FOR: APT
FETCHING DATA FOR: 

FETCHING DATA FOR: HAS
FETCHING DATA FOR: HK
FETCHING DATA FOR: KEK
FETCHING DATA FOR: RILA
FETCHING DATA FOR: RIT
FETCHING DATA FOR: TPET
TRAIN NUMBER 164, 0.328
FETCHING DATA FOR: VNA
FETCHING DATA FOR: VNAT
FETCHING DATA FOR: RPO
FETCHING DATA FOR: PL
FETCHING DATA FOR: R660
FETCHING DATA FOR: NLÄ
FETCHING DATA FOR: OLLI
FETCHING DATA FOR: SLD
TRAIN NUMBER 165, 0.33
FETCHING DATA FOR: KVT
TRAIN NUMBER 166, 0.332
TRAIN NUMBER 167, 0.334
TRAIN NUMBER 168, 0.336
TRAIN NUMBER 169, 0.338
TRAIN NUMBER 170, 0.34
FETCHING DATA FOR: TMU
FETCHING DATA FOR: PMLA
FETCHING DATA FOR: PMT
TRAIN NUMBER 171, 0.342
TRAIN NUMBER 172, 0.344
TRAIN NUMBER 173, 0.346
FETCHING DATA FOR: KUK
TRAIN NUMBER 174, 0.348
TRAIN NUMBER 175, 0.35
TRAIN NUMBER 176, 0.352
TRAIN NUMBER 177, 0.354
FETCHING DATA FOR: OIK
FETCHING DATA FOR: HMA
TRAIN NUMBER 178, 0.356
TRAIN NUMBER 179, 0.358
FETCHING DATA FOR: MSS
FETCHING DATA FOR: KOO
TRAIN NUMBER 180, 0.36
TRAIN NUMBER 181, 0.362
TRAIN NUMBER 182, 0.364
TRAIN NUMBER 18

TRAIN NUMBER 433, 0.866
TRAIN NUMBER 434, 0.868
TRAIN NUMBER 435, 0.87
TRAIN NUMBER 436, 0.872
TRAIN NUMBER 437, 0.874
TRAIN NUMBER 438, 0.876
TRAIN NUMBER 439, 0.878
TRAIN NUMBER 440, 0.88
TRAIN NUMBER 441, 0.882
TRAIN NUMBER 442, 0.884
TRAIN NUMBER 443, 0.886
TRAIN NUMBER 444, 0.888
TRAIN NUMBER 445, 0.89
TRAIN NUMBER 446, 0.892
TRAIN NUMBER 447, 0.894
TRAIN NUMBER 448, 0.896
TRAIN NUMBER 449, 0.898
TRAIN NUMBER 450, 0.9
TRAIN NUMBER 451, 0.902
TRAIN NUMBER 452, 0.904
TRAIN NUMBER 453, 0.906
TRAIN NUMBER 454, 0.908
TRAIN NUMBER 455, 0.91
TRAIN NUMBER 456, 0.912
TRAIN NUMBER 457, 0.914
TRAIN NUMBER 458, 0.916
TRAIN NUMBER 459, 0.918
TRAIN NUMBER 460, 0.92
TRAIN NUMBER 461, 0.922
TRAIN NUMBER 462, 0.924
TRAIN NUMBER 463, 0.926
TRAIN NUMBER 464, 0.928
TRAIN NUMBER 465, 0.93
TRAIN NUMBER 466, 0.932
TRAIN NUMBER 467, 0.934
TRAIN NUMBER 468, 0.936
TRAIN NUMBER 469, 0.938
TRAIN NUMBER 470, 0.94
TRAIN NUMBER 471, 0.942
TRAIN NUMBER 472, 0.944
TRAIN NUMBER 473, 0.946
TRAIN NUMBER 474, 0.948
T